In [1]:
import xgboost as xgb
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PowerTransformer, QuantileTransformer

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
from rdkit import DataStructs, Chem
from rdkit.Chem import AllChem
#import shap
from sklearn.model_selection import cross_validate
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor,AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor,GradientBoostingRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge,ARDRegression,BayesianRidge,ElasticNet,GammaRegressor,HuberRegressor
from sklearn.linear_model import Lasso, LassoLars, LinearRegression, LogisticRegression, PassiveAggressiveRegressor,Ridge,SGDRegressor
import gc
import glob
import os
import random
import time
from datetime import date, datetime

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import shap
from sklearn import model_selection
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_excel('D:/python/rdkit/data2.xlsx')


In [3]:
import random
from rdkit import Chem
aug_data = 300

def randomize_smiles(mol, random_type=None, isomericSmiles=True):
   
    if random_type is None:
        
        random_type = np.random.choice(['unrestricted', 'restricted', 'rotated'])

    if random_type == "unrestricted":
        return Chem.MolToSmiles(mol, canonical=False, doRandom=True, isomericSmiles=isomericSmiles)
    elif random_type == "restricted":
        new_atom_order = list(range(mol.GetNumAtoms()))
        random.shuffle(new_atom_order)
        random_mol = Chem.RenumberAtoms(mol, newOrder=new_atom_order)
        return Chem.MolToSmiles(random_mol, canonical=False, isomericSmiles=isomericSmiles)
    elif random_type == 'rotated':
        n_atoms = mol.GetNumAtoms()
        rotation_index = random.randint(0, n_atoms-1)
        atoms = list(range(n_atoms))
        new_atoms_order = (atoms[rotation_index%len(atoms):]+atoms[:rotation_index%len(atoms)])
        rotated_mol = Chem.RenumberAtoms(mol,new_atoms_order)
        return Chem.MolToSmiles(rotated_mol, canonical=False, isomericSmiles=isomericSmiles)
    raise ValueError("Type '{}' is not valid".format(random_type))

smiles = list(data["a-smile"])
solubilities = list(data["Permeability"])
smilesb = list(data["b-smile"])

aug_smiles=[]
aug_solubilities=[]
aug_smilesb=[]
for sml, sol, sml2 in zip(smiles, solubilities, smilesb):
    
    new_smls = []
    new_smls.append(sml)
    aug_solubilities.append(sol)
    aug_smilesb.append(sml2)
    for _ in range(aug_data):
        
        try:
            new_sml = randomize_smiles(Chem.MolFromSmiles(sml))
            if new_sml not in new_smls:
                new_smls.append(new_sml)
                aug_solubilities.append(sol)
                aug_smilesb.append(sml2)
        except:     
            continue
    aug_smiles.extend(new_smls)

aug_df_AqSolDB = pd.DataFrame(data={
    "SMILES": aug_smiles,
    "Solubility": aug_solubilities,
    "SMILESb": aug_smilesb,
})

print(f"The dataset was augmented from {len(data)} to {len(aug_df_AqSolDB)}.")

The dataset was augmented from 583 to 17523.


In [4]:
aug_df_AqSolDB

,SMILES,Solubility,SMILESb
0,N1CCNCC1,7.5,C
1,C1CNCCN1,7.5,C
2,C1NCCNC1,7.5,C
3,N1CCNCC1,7.0,C
4,C1NCCNC1,7.0,C
...,...,...,...
17518,Oc1c(cc(C(=O)OCC2C(O)C(OC(=O)c3cc(O)c(O)c(O)c3...,12.0,C
17519,c1c(c(c(O)cc1C(OC1C(O)C(C(C(O1)COC(c1cc(O)c(O)...,12.0,C
17520,Oc1cc(C(OC2OC(COC(=O)c3cc(O)c(O)c(O)c3)C(O)C(O...,12.0,C
17521,O=C(c1cc(O)c(c(O)c1)O)OCC1C(C(OC(c2cc(O)c(c(O)...,12.0,C


In [5]:
class morgan_fp:
    def __init__(self, radius, length):
        self.radius = radius
        self.length = length
    def __call__(self, smiles):
        mol = Chem.MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, self.radius, 2048)
        npfp = np.array(list(fp.ToBitString())).astype('float32')
        return npfp

In [6]:
fp = morgan_fp(0,2048)
aug_df_AqSolDB['SMILES'] = aug_df_AqSolDB['SMILES'].apply(fp)
#data['b-fp'] = data['b-smile'].apply(fp)

In [7]:
x_a=np.array(list(aug_df_AqSolDB['SMILES']))


In [8]:
len(x_a)

17523

In [9]:
fp = morgan_fp(0,2048)
aug_df_AqSolDB['SMILESb'] = aug_df_AqSolDB['SMILESb'].apply(fp)

In [10]:
x_b=np.array(list(aug_df_AqSolDB['SMILESb']))

In [11]:
len(x_b)

17523

In [12]:
import numpy as np

# Assuming x_a and x_b are already defined

# Add the arrays element-wise
result = x_a + x_b


In [13]:
len(result)

17523

In [14]:
x=result

In [15]:
y = aug_df_AqSolDB.iloc[:,1]
y

0         7.5
1         7.5
2         7.5
3         7.0
4         7.0
         ... 
17518    12.0
17519    12.0
17520    12.0
17521    12.0
17522    12.0
Name: Solubility, Length: 17523, dtype: float64

In [16]:
import numpy as np

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [18]:
import catboost as cb
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
#import shap
#from matplotlib import pyplot as pltfrom sklearn.datasets 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from catboost import CatBoostClassifier, FeaturesData

In [17]:
model2 = RandomForestRegressor(n_estimators=100, random_state=42)
model2.fit(X_train, Y_train)

RandomForestRegressor(random_state=42)

In [18]:
Y_pred_train = model2.predict(X_train)
#predicts the X_train
#print('Coefficients:', model.coef_)
#print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y_train, Y_pred_train,squared=False))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y_train, Y_pred_train))

Mean squared error (MSE): 5.23
Coefficient of determination (R^2): 0.48


In [19]:
Y_pred_test = model2.predict(X_test)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y_test, Y_pred_test,squared=False))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y_test, Y_pred_test))

Mean squared error (MSE): 5.22
Coefficient of determination (R^2): 0.47


In [20]:
mae = mean_absolute_error(Y_test, Y_pred_test)
mae

3.718931554483071

In [21]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_a)
from scipy.stats import zscore


# Calculate Z-scores for each feature
z_scores = zscore(x_scaled)

# Define a threshold for Z-score to identify outliers
threshold = 3

# Identify outliers
outliers = (abs(z_scores) > threshold).any(axis=1)

# Remove outliers from the feature array and target variable
x_no_outliers = x_a[~outliers]
y_no_outliers = y[~outliers]

# Now you can train your Random Forest model on x_no_outliers and y_no_outliers
#model = RandomForestRegressor()
#model.fit(x_no_outliers, y_no_outliers)

In [22]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Assuming you have x_no_outliers and y_no_outliers from the previous code
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_no_outliers, y_no_outliers, test_size=0.2, random_state=42)

# Initialize and train your Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_test = model.predict(X_test)


print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_test,squared=False))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_test))
# Calculate Mean Squared Error (MSE)
#mse = mean_squared_error(y_test, y_pred)
#print("Mean Squared Error (MSE):", mse)

# Calculate Coefficient of Determination (R^2)
#r2 = r2_score(y_test, y_pred)
#print("Coefficient of Determination (R^2):", r2)


Mean squared error (MSE): 5.17
Coefficient of determination (R^2): 0.51


In [24]:
len(x_no_outliers)

12855

In [23]:
mae = mean_absolute_error(y_test, y_pred_test)
mae

3.7598800906247662